# PII Rule Demonstration Notebook

In this notebook we will walk through the following steps in order to demonstrate what the Shield PII Data Rule is capable of.

Pre  Requisites: 
- A Shield env and API key.

1. Create a new task for PII Rule evaluation 
2. Example dataset evaluation 
   1. Run the examples against a pre-configured Shield task from Step 1 
   2. View our results 

#### Configure Shield Test Env Details

In [13]:
%pip install datasets
%pip install scikit-learn
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from os.path import abspath, join
import sys
from datetime import datetime

utils_path = abspath(join('..', 'utils'))
if utils_path not in sys.path:
    sys.path.append(utils_path)

from shield_utils import setup_env, set_up_task_and_rule, run_shield_evaluation
from analysis_utils import print_performance_metrics, granular_result_dfs


pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

setup_env(base_url="<URL>", api_key="<API_KEY>")


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


'Setup against: <URL>/api/v2'

---
### 1.Setup: Configure a test task and enable Prompt Injection Rule 

In [7]:
pii_rule_no_config = {
    "name": "PII Rule",
    "type": "PIIDataRule",
    "apply_to_prompt": True,
    "apply_to_response": True,
    "config": {}
}

# Create task, archive all rules except the one we pass, create the rule we pass 
pii_rule, pii_task = set_up_task_and_rule(pii_rule_no_config, "pii-task")

print(pii_rule)
print(pii_task)

{'id': 'ed4d9dc7-6d0a-4310-8c54-0444b41ac277', 'name': 'PII Rule', 'type': 'PIIDataRule', 'apply_to_prompt': True, 'apply_to_response': True, 'enabled': True, 'scope': 'task', 'created_at': 1711814624400, 'updated_at': 1711814624400, 'config': {'disabled_pii_entities': [], 'confidence_threshold': 0.0, 'allow_list': []}}
{'id': '5ed22062-febe-4761-9018-b5c9a4b773f4', 'name': 'pii-task-8902', 'created_at': 1711814624312, 'updated_at': 1711814624312, 'rules': [{'id': 'ed4d9dc7-6d0a-4310-8c54-0444b41ac277', 'name': 'PII Rule', 'type': 'PIIDataRule', 'apply_to_prompt': True, 'apply_to_response': True, 'enabled': True, 'scope': 'task', 'created_at': 1711814624400, 'updated_at': 1711814624400, 'config': {'disabled_pii_entities': [], 'confidence_threshold': 0.0, 'allow_list': []}}]}


---
### 2. Examples

In [8]:
pii_examples = pd.read_csv("./datasets/pii_examples.csv")

#### 2.1  Run the examples against a pre-configured Shield task from Step 1 

In [9]:
pii_examples = run_shield_evaluation(pii_examples, pii_task, pii_rule)
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
pii_examples.to_csv(f"./results/pii_examples_df_{current_datetime}.csv")

#### 2.2 Analyze Results

In [10]:
print_performance_metrics(pii_examples)

arthur_fn, arthur_fp, arthur_tp, arthur_tn = granular_result_dfs(pii_examples)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Confusion Matrix:
[[TN FP] 
 [FN TP]] 
[[ 7  0]
 [ 0 21]]


In [12]:
arthur_tp

,Unnamed: 0,text,label,categories,span,shield_result,shield_result_full_detail
0,0,Ada Lovelace was born on 10 December 1815 at 2:34pm,True,"PERSON, DATE_TIME","Ada Lovelace, 10 December 1815",True,"{'inference_id': '49a2aed0-de36-4fa4-9157-1e0094b80ae5', 'rule_results': [{'id': 'ed4d9dc7-6d0a-4310-8c54-0444b41ac277', 'name': 'PII Rule', 'rule_type': 'PIIDataRule', 'scope': 'task', 'result': 'Fail', 'latency_ms': 20, 'details': {'message': 'PII found in data: PERSON,DATE_TIME', 'claims': [], 'pii_results': ['PERSON', 'DATE_TIME'], 'pii_entities': [{'entity': 'PERSON', 'span': 'Ada Lovelace', 'confidence': 0.85}, {'entity': 'DATE_TIME', 'span': '10 December 1815', 'confidence': 0.85}], 'keyword_matches': []}}]}"
1,1,Ada Lovelace's full name is Augusta Ada King,True,"PERSON, PERSON","Ada Lovelace's, Augusta Ada King",True,"{'inference_id': '316c65d5-9ca6-4449-b2c1-baf657227eb7', 'rule_results': [{'id': 'ed4d9dc7-6d0a-4310-8c54-0444b41ac277', 'name': 'PII Rule', 'rule_type': 'PIIDataRule', 'scope': 'task', 'result': 'Fail', 'latency_ms': 13, 'details': {'message': 'PII found in data: PERSON,PERSON', 'claims': [], 'pii_results': ['PERSON', 'PERSON'], 'pii_entities': [{'entity': 'PERSON', 'span': ""Ada Lovelace's"", 'confidence': 0.85}, {'entity': 'PERSON', 'span': 'Augusta Ada King', 'confidence': 0.85}], 'keyword_matches': []}}]}"
2,2,Ada Lovelace was a mathematician and writer,True,PERSON,Ada Lovelace,True,"{'inference_id': '57f1b07f-1a24-4eaa-a97d-e382a8050943', 'rule_results': [{'id': 'ed4d9dc7-6d0a-4310-8c54-0444b41ac277', 'name': 'PII Rule', 'rule_type': 'PIIDataRule', 'scope': 'task', 'result': 'Fail', 'latency_ms': 7, 'details': {'message': 'PII found in data: PERSON', 'claims': [], 'pii_results': ['PERSON'], 'pii_entities': [{'entity': 'PERSON', 'span': 'Ada Lovelace', 'confidence': 0.85}], 'keyword_matches': []}}]}"
4,4,she was a British mathematician and writer,True,NRP,British,True,"{'inference_id': '00e3878b-55a1-4d56-941b-994f31dfd485', 'rule_results': [{'id': 'ed4d9dc7-6d0a-4310-8c54-0444b41ac277', 'name': 'PII Rule', 'rule_type': 'PIIDataRule', 'scope': 'task', 'result': 'Fail', 'latency_ms': 14, 'details': {'message': 'PII found in data: NRP', 'claims': [], 'pii_results': ['NRP'], 'pii_entities': [{'entity': 'NRP', 'span': 'British', 'confidence': 0.85}], 'keyword_matches': []}}]}"
5,5,"she is chiefly known for her work on Charles Babbage's proposed mechanical general-purpose computer, the Analytical Engine",True,PERSON,Charles Babbage's,True,"{'inference_id': 'ba394b96-f3ff-4566-8692-9bd39099af83', 'rule_results': [{'id': 'ed4d9dc7-6d0a-4310-8c54-0444b41ac277', 'name': 'PII Rule', 'rule_type': 'PIIDataRule', 'scope': 'task', 'result': 'Fail', 'latency_ms': 10, 'details': {'message': 'PII found in data: PERSON', 'claims': [], 'pii_results': ['PERSON'], 'pii_entities': [{'entity': 'PERSON', 'span': ""Charles Babbage's"", 'confidence': 0.85}], 'keyword_matches': []}}]}"
6,6,"she died on November 27, 1852",True,DATE_TIME,"November 27, 1852",True,"{'inference_id': 'ff37beb8-13fa-4d50-a3c1-4156daeb332d', 'rule_results': [{'id': 'ed4d9dc7-6d0a-4310-8c54-0444b41ac277', 'name': 'PII Rule', 'rule_type': 'PIIDataRule', 'scope': 'task', 'result': 'Fail', 'latency_ms': 12, 'details': {'message': 'PII found in data: DATE_TIME', 'claims': [], 'pii_results': ['DATE_TIME'], 'pii_entities': [{'entity': 'DATE_TIME', 'span': 'November 27, 1852', 'confidence': 0.85}], 'keyword_matches': []}}]}"
8,8,Her email address is ada.lovelace@gmail.com,True,"EMAIL_ADDRESS, URL","ada.lovelace@gmail.com, gmail.com",True,"{'inference_id': '46db292e-2191-4f1e-b5f2-6ae3ff74ae31', 'rule_results': [{'id': 'ed4d9dc7-6d0a-4310-8c54-0444b41ac277', 'name': 'PII Rule', 'rule_type': 'PIIDataRule', 'scope': 'task', 'result': 'Fail', 'latency_ms': 9, 'details': {'message': 'PII found in data: EMAIL_ADDRESS,URL', 'claims': [], 'pii_results': ['EMAIL_ADDRESS', 'URL'], 'pii_entities': [{'entity': 'EMAIL_ADDRESS', 'span': 'ada.lovelace@gmail.com', 'confidence': 1